# CNC Milling Tool Wear - CaseStudy

## 0. About Dataset

### Context
A series of machining experiments were run on 2" x 2" x 1.5" wax blocks in a CNC milling machine in the System-level Manufacturing and Automation Research Testbed (SMART) at the University of Michigan. Machining data was collected from a CNC machine for variations of tool condition, feed rate, and clamping pressure. Each experiment produced a finished wax part with an "S" shape - S for smart manufacturing - carved into the top face, as shown in test_artifact.jpg (included in the dataset).

### Content
General data from each of the 18 different experiments are given in train.csv and includes the experiment number, material (wax), feed rate, and clamp pressure. Outputs per experiment include tool condition (unworn and worn tools) and whether or not the tool passed visual inspection.

Time series data was collected from the 18 experiments with a sampling rate of 100 ms and are separately reported in files experiment_01.csv to experiment_18.csv. Each file has measurements from the 4 motors in the CNC (X, Y, Z axes and spindle). These CNC measurements can be used in two ways:

1. Taking every CNC measurement as an independent observation where the operation being performed is given in the Machining_Process column. Active machining operations are labeled as "Layer 1 Up", "Layer 1 Down", "Layer 2 Up", "Layer 2 Down", "Layer 3 Up", and "Layer 3 Down".
2. Taking each one of the 18 experiments (the entire time series) as an observation for time series classification

Note that some variables will not accurately reflect the operation of the CNC machine. This can usually be detected by when M1_CURRENT_FEEDRATE reads 50, when X1 ActualPosition reads 198, or when M1_CURRENT_PROGRAM_NUMBER does not read 0. The source of these errors has not been identified.


### Acknowledgements
This data was extracted using the Rockwell Cloud Collector Agent Elastic software from a CNC milling machine in the System-level Manufacturing and Automation Research Testbed (SMART) at the University of Michigan.

### Inspiration

The dataset can be used in classification studies such as:

1. **Tool wear detection** : Supervised binary classification could be performed for identification of worn and unworn cutting tools. Eight experiments were run with an unworn tool while ten were run with a worn tool (see tool_condition column for indication).
2. **Detection of inadequate clamping** : The data could be used to detect when a workpiece is not being held in the vise with sufficient pressure to pass visual inspection (see passed_visual_inspection column for indication of visual flaws). Experiments were run with pressures of 2.5, 3.0, and 4.0 bar. The data could also be used for detecting when conditions are critical enough to prevent the machining operation from completing (see machining_completed column for indication of when machining was preemptively stopped due to safety concerns).

In [1]:
#importing some basic libraries which are commonly used for data science

import pandas as pd    #Pandas Library to work on tabular data
import numpy as np     #Numerical Python Library to use mathematical functions
import matplotlib.pyplot as plt    #Mathematical plotting library
import seaborn as sns    #Visualization Library Seaborn
import os   # OS library to read path of this file, etc.

# Setting options for the Pandas library to display all columns and rows in the dataset if required
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# This line of code is to display the charts within the Jupyter Notebook
%matplotlib inline

General data from a total of 18 different experiments are given in train.csv and includes:

**Inputs (features)**

No : experiment number

*   material : wax
*   feed_rate : relative velocity of the cutting tool along the workpiece (mm/s)
* clamp_pressure : pressure used to hold the workpiece in the vise (bar)

**Outputs (predictions)**

* tool_condition : label for unworn and worn tools
* machining_completed : indicator for if machining was completed without the workpiece moving out of the pneumatic vise
* passed_visual_inspection: indicator for if the workpiece passed visual inspection, only available for experiments where machining was completed

<br>
Time series data was collected from 18 experiments with a sampling rate of 100 ms and are separately reported in files experiment_01.csv to experiment_18.csv. Each file has measurements from the 4 motors in the CNC (X, Y, Z axes and spindle). These CNC measurements can be used in two ways:

1. Taking every CNC measurement as an independent observation where the operation being performed is given in the Machining_Process column. Active machining operations are labeled as "Layer 1 Up", "Layer 1 Down", "Layer 2 Up", "Layer 2 Down", "Layer 3 Up", and "Layer 3 Down".

2. Taking each one of the 18 experiments (the entire time series) as an observation for time series classification


**The features available in the machining datasets are:**
* X1_ActualPosition: actual x position of part (mm)
* X1_ActualVelocity: actual x velocity of part (mm/s)
* X1_ActualAcceleration: actual x acceleration of part (mm/s/s)
* X1_CommandPosition: reference x position of part (mm)
* X1_CommandVelocity: reference x velocity of part (mm/s)
* X1_CommandAcceleration: reference x acceleration of part (mm/s/s)
* X1_CurrentFeedback: current (A)
* X1_DCBusVoltage: voltage (V)
* X1_OutputCurrent: current (A)
* X1_OutputVoltage: voltage (V)
* X1_OutputPower: power (kW)
  
   
* Y1_ActualPosition: actual y position of part (mm)
* Y1_ActualVelocity: actual y velocity of part (mm/s)
* Y1_ActualAcceleration: actual y acceleration of part (mm/s/s)
* Y1_CommandPosition: reference y position of part (mm)
* Y1_CommandVelocity: reference y velocity of part (mm/s)
* Y1_CommandAcceleration: reference y acceleration of part (mm/s/s)
* Y1_CurrentFeedback: current (A)
* Y1_DCBusVoltage: voltage (V)
* Y1_OutputCurrent: current (A)
* Y1_OutputVoltage: voltage (V)
* Y1_OutputPower: power (kW)  
  

* Z1_ActualPosition: actual z position of part (mm)
* Z1_ActualVelocity: actual z velocity of part (mm/s)
* Z1_ActualAcceleration: actual z acceleration of part (mm/s/s)
* Z1_CommandPosition: reference z position of part (mm)
* Z1_CommandVelocity: reference z velocity of part (mm/s)
* Z1_CommandAcceleration: reference z acceleration of part (mm/s/s)
* Z1_CurrentFeedback: current (A)
* Z1_DCBusVoltage: voltage (V)
* Z1_OutputCurrent: current (A)
* Z1_OutputVoltage: voltage (V)
  

* S1_ActualPosition: actual position of spindle (mm)
* S1_ActualVelocity: actual velocity of spindle (mm/s)
* S1_ActualAcceleration: actual acceleration of spindle (mm/s/s)
* S1_CommandPosition: reference position of spindle (mm)
* S1_CommandVelocity: reference velocity of spindle (mm/s)
* S1_CommandAcceleration: reference acceleration of spindle (mm/s/s)
* S1_CurrentFeedback: current (A)
* S1_DCBusVoltage: voltage (V)
* S1_OutputCurrent: current (A)
* S1_OutputVoltage: voltage (V)
* S1_OutputPower: current (A)
* S1_SystemInertia: torque inertia (kg*m^2)
  

* M1_CURRENT_PROGRAM_NUMBER: number the program is listed under on the CNC
* M1_sequence_number: line of G-code being executed
* M1_CURRENT_FEEDRATE: instantaneous feed rate of spindle
  


**Machining_Process:** the current machining stage being performed. Includes preparation, tracing up  and down the "S" curve involving different layers, and repositioning of the spindle as it moves through the air to a certain starting point


**Note:** Some variables will not accurately reflect the operation of the CNC machine. This can usually be detected by when M1_CURRENT_FEEDRATE reads 50, when X1 ActualPosition reads 198, or when M1_CURRENT_PROGRAM_NUMBER does not read 0. The source of these errors has not been identified.


## 1. Data Collection : Importing the data

In [3]:
# importing the data from Kaggle
# We will simply downlaod the dataset from the Kaggle website and place it in an accessible location
# https://www.kaggle.com/datasets/shasun/tool-wear-detection-in-cnc-mill?resource=download

#Running these below statement to mount my Google Drive on Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = "/content/drive/MyDrive/Work/RevaCollegePresentation/CNC_Milling_Dataset"

In [5]:
from pathlib import Path
df_list=[]
for path in Path(data_path).iterdir():
    if path.suffix ==".csv" and path.name!="train.csv":
        exp_data = pd.read_csv(str(path))
        df_list.append(exp_data)
all_exp_data = pd.concat(df_list)

(25286, 48)


In [10]:
print(all_exp_data.shape)

(25286, 48)


In [6]:
#let's look at a sample of the data
all_exp_data.head()  #shows top 5 rows of the data

,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,X1_OutputPower,Y1_ActualPosition,Y1_ActualVelocity,Y1_ActualAcceleration,Y1_CommandPosition,Y1_CommandVelocity,Y1_CommandAcceleration,Y1_CurrentFeedback,Y1_DCBusVoltage,Y1_OutputCurrent,Y1_OutputVoltage,Y1_OutputPower,Z1_ActualPosition,Z1_ActualVelocity,Z1_ActualAcceleration,Z1_CommandPosition,Z1_CommandVelocity,Z1_CommandAcceleration,Z1_CurrentFeedback,Z1_DCBusVoltage,Z1_OutputCurrent,Z1_OutputVoltage,S1_ActualPosition,S1_ActualVelocity,S1_ActualAcceleration,S1_CommandPosition,S1_CommandVelocity,S1_CommandAcceleration,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process
0,141.0,-0.200,-100.0,141.0,0.0,0.0,-2.450,0.0512,327.0,2.69,0.000030,96.6,6.05,-75.0,96.7,6.0,0.0,2.72,0.0578,326.0,12.7,0.000714,29.7,0.025,6.25,29.7,0.0,0.0,0.0,0.0,0.0,0.0,-1540.0,53.4,45.30,-1540.0,53.3,0.000000e+00,18.7,0.765,323.0,115.0,0.145,12.0,1.0,2.0,6.0,Prep
1,141.0,-0.300,-144.0,141.0,0.0,0.0,-1.400,0.0479,327.0,3.18,0.000014,97.2,6.28,18.8,97.3,6.0,0.0,2.93,0.0756,326.0,12.4,0.000893,29.7,0.025,12.50,29.7,0.0,0.0,0.0,0.0,0.0,0.0,-1530.0,53.3,11.90,-1530.0,53.3,0.000000e+00,24.8,1.000,322.0,117.0,0.193,12.0,1.0,26.0,6.0,Prep
2,141.0,-0.250,-131.0,141.0,0.0,0.0,-2.290,0.0483,327.0,2.10,0.000015,97.8,6.10,68.7,97.9,6.0,0.0,4.96,0.1080,325.0,14.0,0.001380,29.7,0.025,12.50,29.7,0.0,0.0,0.0,0.0,0.0,0.0,-1530.0,53.4,14.10,-1530.0,53.3,0.000000e+00,17.8,0.881,323.0,114.0,0.170,12.0,1.0,26.0,6.0,Prep
3,141.0,-0.175,-93.7,141.0,0.0,0.0,-0.851,0.0488,327.0,4.08,0.000007,98.4,5.97,-12.5,98.5,6.0,0.0,5.71,0.1140,326.0,14.8,0.001470,29.7,-0.025,-6.25,29.7,0.0,0.0,0.0,0.0,0.0,0.0,-1520.0,53.3,-16.10,-1520.0,53.3,9.540000e-07,21.5,0.751,323.0,117.0,0.144,12.0,1.0,26.0,6.0,Prep
4,141.0,-0.200,-106.0,141.0,0.0,0.0,-2.290,0.0502,327.0,2.17,0.000015,99.0,5.93,-75.0,99.1,6.0,0.0,6.06,0.1080,325.0,14.5,0.001300,29.7,0.000,0.00,29.7,0.0,0.0,0.0,0.0,0.0,0.0,-1520.0,53.3,-7.06,-1520.0,53.3,-9.540000e-07,21.6,0.989,322.0,115.0,0.190,12.0,1.0,26.0,6.0,Prep


In [8]:
all_exp_data.tail()   #shows last 5 rows of the data

,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,X1_OutputPower,Y1_ActualPosition,Y1_ActualVelocity,Y1_ActualAcceleration,Y1_CommandPosition,Y1_CommandVelocity,Y1_CommandAcceleration,Y1_CurrentFeedback,Y1_DCBusVoltage,Y1_OutputCurrent,Y1_OutputVoltage,Y1_OutputPower,Z1_ActualPosition,Z1_ActualVelocity,Z1_ActualAcceleration,Z1_CommandPosition,Z1_CommandVelocity,Z1_CommandAcceleration,Z1_CurrentFeedback,Z1_DCBusVoltage,Z1_OutputCurrent,Z1_OutputVoltage,S1_ActualPosition,S1_ActualVelocity,S1_ActualAcceleration,S1_CommandPosition,S1_CommandVelocity,S1_CommandAcceleration,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process
2327,165.0,-17.9,37.50,165.0,-17.9,-0.000095,-5.43,0.108,329.0,28.3,0.00408,98.6,-32.3,-62.5,98.3,-32.4,0.000000,-11.40,0.190,326.0,49.5,0.0131,57.2,-33.6,-12.50,57.0,-33.6,0.0,0.0,0.0,0.0,0.0,42.0,0.000,-0.25,42.0,0.0,0.0,0.40800,2.700000e-19,329.0,0.0,-0.000001,12.0,1.0,7.0,50.0,End
2328,163.0,-18.1,-6.25,163.0,-17.9,0.000000,-4.77,0.107,329.0,30.0,0.00389,95.3,-32.6,-87.5,95.1,-32.4,0.000000,-11.20,0.177,326.0,49.2,0.0120,53.9,-33.7,-18.80,53.6,-33.6,0.0,0.0,0.0,0.0,0.0,42.0,0.000,-0.25,42.0,0.0,0.0,0.00499,2.700000e-19,329.0,0.0,-0.000002,12.0,1.0,7.0,50.0,End
2329,162.0,-18.1,-81.20,161.0,-17.9,0.000000,-5.61,0.104,328.0,30.3,0.00387,92.1,-32.3,50.0,91.8,-32.4,-0.000095,-8.07,0.185,326.0,50.7,0.0128,50.5,-33.6,25.00,50.3,-33.6,0.0,0.0,0.0,0.0,0.0,42.0,-0.002,-0.50,42.0,0.0,0.0,-0.08760,2.700000e-19,329.0,0.0,0.000001,12.0,1.0,7.0,50.0,End
2330,160.0,-18.0,-6.25,160.0,-17.9,0.000000,-7.79,0.134,328.0,30.3,0.00508,88.9,-32.6,-62.5,88.6,-32.4,0.000095,-6.61,0.154,326.0,49.3,0.0106,47.2,-33.6,18.80,46.9,-33.6,0.0,0.0,0.0,0.0,0.0,42.0,0.002,0.50,42.0,0.0,0.0,-0.27300,2.700000e-19,329.0,0.0,-0.000003,12.0,1.0,7.0,50.0,End
2331,158.0,-17.8,25.00,158.0,-17.9,0.000191,-8.14,0.138,328.0,29.6,0.00521,85.6,-32.3,62.5,85.4,-32.4,0.000000,-9.99,0.162,326.0,51.0,0.0109,43.8,-33.8,6.25,43.5,-33.6,0.0,0.0,0.0,0.0,0.0,42.0,0.001,0.50,42.0,0.0,0.0,-0.47500,2.700000e-19,329.0,0.0,-0.000002,12.0,1.0,7.0,50.0,End


In [9]:
all_exp_data.sample(10)   #shows sample n rows of the data

,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,X1_OutputPower,Y1_ActualPosition,Y1_ActualVelocity,Y1_ActualAcceleration,Y1_CommandPosition,Y1_CommandVelocity,Y1_CommandAcceleration,Y1_CurrentFeedback,Y1_DCBusVoltage,Y1_OutputCurrent,Y1_OutputVoltage,Y1_OutputPower,Z1_ActualPosition,Z1_ActualVelocity,Z1_ActualAcceleration,Z1_CommandPosition,Z1_CommandVelocity,Z1_CommandAcceleration,Z1_CurrentFeedback,Z1_DCBusVoltage,Z1_OutputCurrent,Z1_OutputVoltage,S1_ActualPosition,S1_ActualVelocity,S1_ActualAcceleration,S1_CommandPosition,S1_CommandVelocity,S1_CommandAcceleration,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process
450,141.0,0.125,75.00,141.0,0.000,0.000,-2.8400,5.100000e-02,329.0,6.990,-0.000045,77.8,-0.025,-6.25,77.8,0.0000,0.000,0.00265,2.370000e-02,328.0,3.160,0.000000e+00,55.5,0.000,0.00,55.5,0.0,0.0,0.0,0.0,0.0,0.0,1550.0,1.410,-27.90,1550.0,0.0,0.000000e+00,0.983,2.770000e-19,329.0,0.0,0.00011,12.0,1.0,0.0,50.0,Layer 1 Down
1802,151.0,-0.100,-43.80,151.0,0.000,0.000,1.4100,2.400000e-02,328.0,1.600,-0.000010,73.0,-0.075,-31.30,73.0,0.0000,0.000,1.02000,2.010000e-02,326.0,1.530,-4.380000e-07,38.7,0.225,25.00,38.7,-3.0,-281.0,0.0,0.0,0.0,0.0,-1070.0,53.200,-46.30,-1070.0,53.3,0.000000e+00,28.300,9.780000e-01,321.0,119.0,0.18500,12.0,1.0,94.0,3.0,Layer 3 Down
1625,162.0,0.150,81.20,162.0,0.000,0.000,3.8800,6.570000e-02,327.0,1.490,0.000020,88.9,-3.280,-81.20,88.9,-3.0000,0.000,-3.02000,5.290000e-02,326.0,6.620,3.500000e-04,29.5,0.000,0.00,29.5,0.0,0.0,0.0,0.0,0.0,0.0,-1510.0,53.400,54.00,-1510.0,53.3,0.000000e+00,17.800,7.350000e-01,322.0,120.0,0.13800,12.0,1.0,2.0,3.0,Layer 3 Up
451,198.0,0.000,0.00,198.0,0.000,0.000,0.6600,2.190000e-02,328.0,1.470,-0.000001,158.0,0.025,12.50,158.0,0.0000,0.000,-0.29300,1.720000e-02,326.0,0.797,0.000000e+00,119.0,0.000,0.00,119.0,0.0,0.0,0.0,0.0,0.0,0.0,-1510.0,0.002,0.25,-1510.0,0.0,0.000000e+00,0.144,2.760000e-19,327.0,0.0,0.00000,12.0,1.0,0.0,6.0,End
931,160.0,0.800,-6.25,160.0,6.000,125.000,15.3000,2.640000e-01,327.0,14.400,0.001990,105.0,0.000,0.00,105.0,-0.0962,-14.100,0.45900,1.780000e-02,325.0,1.380,7.480000e-07,27.7,0.000,0.00,27.7,0.0,0.0,0.0,0.0,0.0,0.0,-1870.0,53.200,-40.40,-1870.0,53.3,0.000000e+00,26.000,9.590000e-01,323.0,121.0,0.18300,12.0,1.0,112.0,6.0,Layer 3 Up
39,144.0,-2.800,-6.25,144.0,-2.720,0.792,-4.9700,1.040000e-01,327.0,9.400,0.000636,73.4,1.200,-37.50,73.4,1.2600,1.720,5.07000,7.300000e-02,326.0,5.720,2.070000e-04,28.7,0.000,6.25,28.7,0.0,0.0,0.0,0.0,0.0,0.0,1690.0,53.300,-27.40,1690.0,53.3,0.000000e+00,26.000,9.750000e-01,320.0,115.0,0.18800,12.0,1.0,56.0,3.0,Prep
918,198.0,0.000,0.00,198.0,0.000,0.000,0.0446,2.790000e-19,328.0,0.000,0.000002,158.0,0.000,0.00,158.0,0.0000,0.000,0.83500,2.680000e-19,332.0,0.000,0.000000e+00,119.0,0.000,6.25,119.0,0.0,0.0,0.0,0.0,0.0,0.0,-1160.0,0.001,0.25,-1160.0,0.0,0.000000e+00,-1.800,0.000000e+00,331.0,0.0,0.00000,12.0,4.0,0.0,50.0,Layer 2 Up
793,159.0,-0.500,-75.00,159.0,-0.461,-9.050,-2.8600,4.260000e-02,327.0,2.290,0.000023,85.8,6.150,100.00,85.9,5.9800,-0.669,7.79000,1.690000e-01,326.0,16.200,2.290000e-03,27.7,-0.025,-6.25,27.7,0.0,0.0,0.0,0.0,0.0,0.0,1960.0,53.500,64.00,1960.0,53.3,0.000000e+00,13.400,7.330000e-01,324.0,116.0,0.14000,12.0,1.0,2.0,6.0,Repositioning
1207,154.0,-2.750,68.70,154.0,-3.000,0.000,-4.1900,8.450000e-02,327.0,10.200,0.000456,72.4,0.050,25.00,72.4,0.0000,0.000,-5.04000,4.460000e-02,326.0,3.010,0.000000e+00,29.5,-0.025,-6.25,29.5,0.0,0.0,0.0,0.0,0.0,0.0,1100.0,53.300,-12.30,1100.0,53.3,-9.540000e-07,20.800,9.220000e-01,320.0,121.0,0.17800,12.0,1.0,46.0,3.0,Layer 2 Up
50,198.0,0.025,0.00,198.0,0.000,0.000,-0.9380,3.050000e-02,329.0,0.159,0.000004,158.0,0.000,0.00,158.0,0.0000,0.000,0.22000,1.590000e-02,327.0,0.658,1.460000e-06,119

In [13]:
all_exp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25286 entries, 0 to 25285
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   X1_ActualPosition          25286 non-null  float64
 1   X1_ActualVelocity          25286 non-null  float64
 2   X1_ActualAcceleration      25286 non-null  float64
 3   X1_CommandPosition         25286 non-null  float64
 4   X1_CommandVelocity         25286 non-null  float64
 5   X1_CommandAcceleration     25286 non-null  float64
 6   X1_CurrentFeedback         25286 non-null  float64
 7   X1_DCBusVoltage            25286 non-null  float64
 8   X1_OutputCurrent           25286 non-null  float64
 9   X1_OutputVoltage           25286 non-null  float64
 10  X1_OutputPower             25286 non-null  float64
 11  Y1_ActualPosition          25286 non-null  float64
 12  Y1_ActualVelocity          25286 non-null  float64
 13  Y1_ActualAcceleration      25286 non-null  flo

In [12]:
all_exp_data.reset_index(inplace=True,drop=True)

## 2. Data Preparation - Data Cleaning & Exploratory Data Analysis

## 3. Model Training & Selection

## 4. Model Evaluation

## 5. Hyperparameter Tuning

## 6. Production Deployment : Serving the model

## 7. Pipeline Monitoring & Maintenance


* Before the model is deployed into production, there should be ample test-cases that should be deployed and alerts should be set-up at all important points of failure.
  * For example, say we are expecting 100-120 units to be manufactured per hour in an assembly line
  * We notice that the output is less than 80, this should send out an alert to monitor for potential failure
  * Or if the unit is producing say 150 units an hour, this hsould also be alerted. These checks could really help with the QA/QC in the manufacturing process
  * Such checks should be set-up where ever possible in order to reduce losses   
* Once the model is deployed in production, quality of the output should be checked at regular intervals and the model should also be refreshed with new available training data in order to capture more cases of Tool Wear and improve the prediction accuracy



## Summary


*   In summary we are now able to predict the tool wear using the CNC Milling dataset.
*   We can now apply similar ML techniques to the one that we have learnt today to various problems in different field of engineering.
* We have learnt to import the data, clean it and build a model to predict tool wear based on the historical data.
* We have also learnt to save the model and predict tool wear based on new incoming data using the above model.



## THANK YOU!